## Домашнее задание к занятию "A/B-тесты"

### Описание задачи

![banner](https://storage.googleapis.com/kaggle-datasets-images/635/1204/126be74882028aac7241553cef0e27a7/dataset-original.jpg)

Покемоны - это маленькие существа, которые сражаются друг с другом на соревнованиях. Все покемоны имеют разные характеристики (сила атаки, защиты и т. д.) И относятся к одному или двум так называемым классам (вода, огонь и т. д.).
Профессор Оук является изобретателем Pokedex - портативного устройства, которое хранит информацию обо всех существующих покемонах. Как его ведущий специалист по данным, Вы только что получили от него запрос с просьбой осуществить аналитику данных на всех устройствах Pokedex.

### Описание набора данных
Профессор Оук скопировал все содержимое в память одного устройства Pokedex, в результате чего получился набор данных, с которым Вы будете работать в этой задаче. В этом файле каждая строка представляет характеристики одного покемона:

* `pid`: Numeric - ID покемона
* `HP`: Numeric - Очки здоровья
* `Attack`: Numeric - Сила обычной атаки
* `Defense`: Numeric - Сила обычной защиты
* `Sp. Atk`: Numeric - Сила специальной атаки
* `Sp. Def`: Numeric - Сила специальной защиты
* `Speed`: Numeric - Скорость движений
* `Legendary`: Boolean - «True», если покемон редкий
* `Class 1`: Categorical - Класс покемона
* `Class 2`: Categorical - Класс покемона

In [6]:
import warnings
# Отключение предупреждений (warnings)
warnings.filterwarnings("ignore")

import pandas as pd

from scipy.stats import ttest_ind
from scipy.stats import f_oneway, shapiro

pokemon = pd.read_csv('pokemon.csv', sep = ',')  # Откроем датасет
pokemon.head()

# Обратите внимание, что у покемона может быть один или два класса.
# Если у покемона два класса, считается, что они имеют одинаковую значимость.

,pid,Name,Class 1,Class 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Legendary
0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,False
1,2,Ivysaur,Grass,Poison,60,62,63,80,80,60,False
2,3,Venusaur,Grass,Poison,80,82,83,100,100,80,False
3,4,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,False
4,5,Charmander,Fire,NaN,39,52,43,60,50,65,False


### Задачи

<div class="alert alert-info">
<b>Задание № 1:</b>
    
Профессор Оук подозревает, что покемоны в классе `Grass` имеют более сильную обычную атаку, чем покемоны в классе `Rock`. Проверьте, прав ли он, и убедите его в своём выводе статистически.
    
    
Примечание: если есть покемоны, которые относятся к обоим классам, просто выбросьте их;
    
Вы можете предположить, что распределение обычных атак является нормальным для всех классов покемонов.

</div>

In [11]:
pokemon.columns = pokemon.columns.str.replace(' ', '_')
pokemon.head()

,pid,Name,Class_1,Class_2,HP,Attack,Defense,Sp._Atk,Sp._Def,Speed,Legendary
0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,False
1,2,Ivysaur,Grass,Poison,60,62,63,80,80,60,False
2,3,Venusaur,Grass,Poison,80,82,83,100,100,80,False
3,4,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,False
4,5,Charmander,Fire,NaN,39,52,43,60,50,65,False


In [29]:
pokemon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   pid        800 non-null    int64 
 1   Name       799 non-null    object
 2   Class_1    800 non-null    object
 3   Class_2    414 non-null    object
 4   HP         800 non-null    int64 
 5   Attack     800 non-null    int64 
 6   Defense    800 non-null    int64 
 7   Sp._Atk    800 non-null    int64 
 8   Sp._Def    800 non-null    int64 
 9   Speed      800 non-null    int64 
 10  Legendary  800 non-null    bool  
dtypes: bool(1), int64(7), object(3)
memory usage: 63.4+ KB


In [20]:
# Фильтрация покемонов только либо в 1 классе, либо в др
grass = (pokemon['Class_1'] == 'Grass') | (pokemon['Class_2'] == 'Grass')
rock = (pokemon['Class_1'] == 'Rock') | (pokemon['Class_2'] == 'Rock')

# Фильтрация покемонов, которые одновременно имеют оба классе
both_types = (
    (pokemon['Class_1'] == 'Grass') & (pokemon['Class_2'] == 'Rock')
) | (
    (pokemon['Class_1'] == 'Rock') & (pokemon['Class_2'] == 'Grass')
)

# Теперь оставляем тольео тех, кто имеет класс либо Grass, либо Rock
grass_only = pokemon[grass & ~both_types]
rock_only = pokemon[rock & ~both_types]

# Выбираем значения Attack
grass_attack = grass_only['Attack'].dropna()
rock_attack = rock_only['Attack'].dropna()

# Выполняем t-тест
t_stat, p_value = ttest_ind(grass_attack, rock_attack, equal_var=True)

if p_value < 0.05:
    print("Покемоны типа Grass имеют статистически значимую разницу в обычной атаке в отличие от типа Rock.")
else:
    print("Нет статистически значимой разницы в обычной атаке между Grass и Rock.")

Профессор Оук прав. Покемоны типа Grass имеют статистически значимо более высокую обычную атаку, чем Rock.


<div class="alert alert-info">
<b>Задание № 2:</b>
    
Профессор Оук уже долго не может спать по ночам, ведь его волнует вопрос, а правда ли, что покемоны в классе `Water` в среднем быстрее, чем покемоны в классе `Normal`.
    
    
Проверьте, прав ли он, и убедите его в своём выводе статистически.
    
Примечание: если есть покемоны, которые относятся к обоим классам, выбросьте их;
    
Вы можете предположить, что распределение скорости движения является нормальным для всех классов покемонов.
</div>

In [24]:
# Фильтрация покемонов только либо в 1 классе, либо в др
water = (pokemon['Class_1'] == 'Water') | (pokemon['Class_2'] == 'Water')
normal = (pokemon['Class_1'] == 'Normal') | (pokemon['Class_2'] == 'Normal')

# Фильтрация покемонов, которые одновременно имеют оба класса
both_types = (
    (pokemon['Class_1'] == 'Water') & (pokemon['Class_2'] == 'Normal')
) | (
    (pokemon['Class_1'] == 'Normal') & (pokemon['Class_2'] == 'Water')
)

# Теперь оставляем только тех, кто имеет класс либо Water, либо Normal
water_only = pokemon[water & ~both_types]
normal_only = pokemon[normal & ~both_types]

# Выбираем значения Attack
water_speed = water_only['Speed'].dropna()
normal_speed = normal_only['Speed'].dropna()

# Выполняем t-тест
t_stat, p_value = ttest_ind(water_speed, normal_speed, equal_var=True)

if p_value < 0.05:
    print("Покемоны типа Water имеют статистически значимую разницу в скорости в отличие от типа Normal.")
else:
    print("Нет статистически значимой разницы в скорости между Water и Normal.")

Покемоны типа Water в среднем более быстрые, чем Normal.


<div class="alert alert-info">
<b>Задание № 3:</b>
    
Профессор Оук тот еще безумец. Он изобрёл сыворотку, способную ускорить покемона. Однако мы усомнились в эффективности его вакцины. Професоор дал эту сыворотку следующим покемонам: смотри массив `treathed_pokemon`. Проверьте, работает ли вообще его сыворотка, убедите всех в своём выводе статистически.
    
    
Вы можете предположить, что распределение скорости движения является нормальным для всех классов покемонов.

</div>

In [25]:
# Покемоны, которые принимали сыворотку увеличения скорости
treathed_pokemon = ['Mega Beedrill', 'Mega Alakazam',
                    'Deoxys Normal Forme', 'Mega Lopunny']

In [65]:
# Фильтруем строки с этими покемонами
treated_data = pokemon[pokemon['Name'].isin(treathed_pokemon)]

# Выбираем скорости покемонов, получивших сыворотку
treated_speed = treated_data['Speed'].dropna()

# Выбираем скорости всех остальных покемонов
all_others = pokemon[~pokemon['Name'].isin(treathed_pokemon)]
others_speed = all_others['Speed'].dropna()

# Выполняем t-тест
t_stat, p_value = ttest_ind(others_speed, treated_speed, equal_var=True)

if p_value < 0.05:
    print('Есть статистически значимая разница в скорости покемонов')
else:
    print('Сыворотка не работает, скорость покемонов не отличается')

,pid,Name,Class_1,Class_2,HP,Attack,Defense,Sp._Atk,Sp._Def,Speed,Legendary


<div class="alert alert-info">
<b>Задание № 4:</b>
    
Профессор Оук всегда любил истории про легендарных покемонов. Однако профессор не очень уверен, что они лучше остальных покемонов. Оук предложил разобраться в этом нам. Проверьте, действительно ли сумма характеристик `HP`,`Attack`,`Defense` у легендарных покемонов выше, чем у других покемонов?

А произведение этих же параметров?

Найдите ответы на эти вопросы и убедите всех в своём выводе статистически.
   

Вы можете предположить, что распределение сум и произведений этих параметров является нормальным для всех классов покемонов.

</div>

In [49]:
# Фильтруем покемонов, на ленегдарных и всеъ остальных
legendary_pokemons = pokemon.loc[pokemon['Legendary'] == True].dropna()
others_pokemons = pokemon.loc[pokemon['Legendary'] == False].dropna()

# Считаем сумму хар-к для 2-х групп покемонов
legendary_pokemons['sum'] = legendary_pokemons['HP'] + legendary_pokemons['Attack'] + legendary_pokemons['Defense']
others_pokemons['sum'] = others_pokemons['HP'] + others_pokemons['Attack'] + others_pokemons['Defense']

# Считаем произведение хар-к для 2-х групп покемонов
legendary_pokemons['product'] = legendary_pokemons['HP'] * legendary_pokemons['Attack'] * legendary_pokemons['Defense']
others_pokemons['product'] = others_pokemons['HP'] * others_pokemons['Attack'] * others_pokemons['Defense']

# T-тест для суммы 
stat_sum, p_val_sum = ttest_ind(legendary_pokemons['sum'], others_pokemons['sum'])

if p_val_sum < 0.05:
    print('Сумма между данными характеристиками у легендарных и обычных покемонов различается')
else:
    print('Сумма между данными характеристиками у легендарных и обычных покемонов не различается')

# T-тест для произведения
stat_product, p_product = ttest_ind(legendary_pokemons['product'], others_pokemons['product'])

if p_product < 0.05:
    print('Произведение между данными характеристиками у легендарных и обычных покемонов различается')
else:
    print('Произведение между данными характеристиками у легендарных и обычных покемонов не различается')

Сумма между данными характеристиками у легендарных и обычных покемонов различается
Произведение между данными характеристиками у легендарных и обычных покемонов различается


<div class="alert alert-info">
<b>Задание № 5:</b>
    
Профессор Оук частенько наблюдает за боями покемонов. После очередных таких боёв Оук выделил четыре класса `best_defence_class`, которые на его взгляд одинаковы по "силе обычной защиты" `Defense`.

Проверьте, действительно ли эти классы покемонов не отличаются по уровню защиты статистически значимо? Всё та же статистика вам в помощь!
   

Вы можете предположить, что распределение параметров защитных характеристик является нормальным для всех классов покемонов.

</div>

In [51]:
best_defence_class = ['Rock', 'Ground', 'Steel', 'Ice']
best_defence_class

['Rock', 'Ground', 'Steel', 'Ice']

In [67]:
# Фильтруем покемонов, у которых хотя бы один из классов входит в best_defence_class
mask = (pokemon['Class_1'].isin(best_defence_class)) | (pokemon['Class_2'].isin(best_defence_class))
filtered_pokemon = pokemon[mask].dropna(subset=['Defense'])

# Разделяем на 4 группы
groups = []
for cls in best_defence_class:
    # Покемоны, у которых Class_1 или Class_2 == cls
    class_mask = (filtered_pokemon['Class_1'] == cls) | (filtered_pokemon['Class_2'] == cls)
    group_defense = filtered_pokemon[class_mask]['Defense'].dropna()
    groups.append(group_defense)

# Проводим дисперсионный анализ
stat, p_value = f_oneway(*groups)

if p_value < 0.05:
    print('Есть статистически значимые различия в уровне защиты между классами.')
else:
    print('Нет статистически значимых различий в уровне защиты между классами.')

Есть статистически значимые различия в уровне защиты между классами.


# **Примечание:**

Домашнее задание сдается ссылкой [Google Colab](https://colab.research.google.com/). Мы не сможем проверить его или помочь, если вы пришлете:

*   файлы;
*   архивы;
*   скриншоты кода.

Все обсуждения и консультации по выполнению домашнего задания ведутся только на соответствующем канале в Discord.

**Как правильно задавать вопросы аспирантам, преподавателям и коллегам:**

Прежде чем задать вопрос, попробуйте найти ответ в интернете. Навык самостоятельного поиска информации — один из важнейших. Каждый практикующий специалист любого уровня делает это ежедневно.

Сформулируйте вопрос по алгоритму:

1.   Что я делаю?
2.   Какого результата я ожидаю?
3.   Как фактический результат отличается от ожидаемого?
4.   Что я уже попробовал сделать, чтобы исправить проблему?

По возможности прикрепите к вопросу скриншоты либо ссылки на код. Не выкладывайте все решение, оставляйте только проблемный и воспроизводимый участок кода.